In [1]:
import subprocess
import numpy as np

In [2]:
# Variable params
# EG scenarios presumed to start in 2022 with first LWR decom in 2023.
intro_time = 12 * 5 # intro time for additive: 5 yrs
sim_duration = 12 * 100 # 100 years (don't need this length for EG01, but prob do for transitions)
trans_time = 12 * 15 # 15 yrs, transition time for EG01 --> EGXX. Transition time is assumed to always be greater than intro time

# "Non-Variable" Params
n_init_rxtrs = 100
n_assem_core = 3
assem_size = 29565
init_total_assem_size = n_init_rxtrs * n_assem_core * assem_size
cycle_time = 18
rxtr_life = 12 * 60 

# Protoype Definitions:
1. EG01 Fuel Cycle Facilities
  - Full Additive Availability
  - Ramp-up Additive Approach
2. EG23 Fuel Cycle Facilities

## 1. EG01 Fuel Cycle Facilities
### (a) Full Additive Availability

In [3]:
init_uox_source = {'name' : 'SourceInitUOX', 
                   'config' : {'Source' : {'outcommod' : 'InitUOX',
                                           'outrecipe' : 'UOX_no232',
                                           'inventory_size' : init_total_assem_size
                                          }
                              }
                  }

In [4]:
from eg01_facilities import (natu_source, non_source, add_source, enrich, 
                             du_store, store_no232, mix_no232, store_232, 
                             mix_232, lwr_cool, lwr_store, sink)

In [5]:
# LWR prototype for full additive availability upon introduction
lwr_full = {'name' : 'LWR',
            'lifetime' : rxtr_life,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['UOX_Add', 'UOX_Non', 'InitUOX']},
                                     'fuel_outcommods' : {'val' : ['SpentUOX_Add', 'SpentUOX_Non', 'SpentUOX_Non']},
                                     'fuel_inrecipes' :  {'val' : ['UOX_232', 'UOX_no232', 'UOX_no232']},
                                     'fuel_outrecipes' : {'val' : ['SpentUOX_232', 'SpentUOX_no232', 'SpentUOX_no232']},
                                     'fuel_prefs' : {'val' : [1, 2, 2.5]},
                                     'pref_change_times' : {'val' : intro_time},
                                     'pref_change_commods' : {'val' : 'UOX_Add'},
                                     'pref_change_values' : {'val' : 3},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_name' : 'PowerLWR',
                                     'power_cap' : 1000
                                    }
                       }
           }

### (b) Additional Facilities for Ramp-up Add Approach

In [ ]:
from eg01_facilities import (store_pct_no232, store_pct_232, 
                             mix_50pct232, mix_10pct232)

# LWR prototype for partial additive availability/slow utility uptake upon introduction
intro_50 = intro_time + 3 * cycle_time
intro_100 = intro_50 + 3 * cycle_time
lwr_ramp = {'name' : 'LWR',
            'lifetime' : 960,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['UOX_Add', 'UOX_50pctAdd', 'UOX_10pctAdd', 'UOX_Non', 'InitUOX']},
                                     'fuel_outcommods' : {'val' : ['SpentUOX_Add', 'SpentUOX_50pctAdd', 'SpentUOX_10pctAdd', 'SpentUOX_Non', 'SpentUOX_Non']},
                                     'fuel_inrecipes' :  {'val' : ['UOX_232', 'UOX_50pct232', 'UOX_10pct232', 'UOX_no232', 'UOX_no232']},
                                     'fuel_outrecipes' : {'val' : ['SpentUOX_232', 'SpentUOX_50pct232', 'SpentUOX_10pct232', 'SpentUOX_no232', 'SpentUOX_no232']},
                                     'fuel_prefs' : {'val' : [1, 1, 1, 2, 2.5]},
                                     'pref_change_times' : {'val' : [intro_time, intro_50, intro_100]},
                                     'pref_change_commods' : {'val' : ['UOX_10pctAdd', 'UOX_50pctAdd', 'UOX_Add']},
                                     'pref_change_values' : {'val' : [3, 4, 5]},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_cap' : 1000
                                    }
                       }
           }

## 2. EG23 Fuel Cycle Facilities

In [6]:
eg23_sink = {'name' : 'Waste',
             'config' : {'Sink' : {'in_commods' : {'val' : 'Waste'}
                                  }
                        }
            }

# Post-LWR Facilities

# Storage facilities can take many inputs but only give one output, so unlike in EG01 
# am now separating the spent fuel cooling and storage into separate streams
non_lwr_cool = {'name' : 'UOXCoolNon',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentUOX_Non'},
                                         'out_commods' : {'val' : 'CooledSpentUOX_Non'},
                                         'in_recipe' : 'SpentUOX_no232',
                                         'residence_time' : 81
                                        }
                           }
               }
add_lwr_cool = {'name' : 'UOXCoolAdd',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentUOX_Add'},
                                         'out_commods' : {'val' : 'CooledSpentUOX_Add'},
                                         'in_recipe' : 'SpentUOX_232',
                                         'residence_time' : 81
                                        }
                           }
               }
lwr_sep_streams = {'item' : [{'commod' : 'UOX_Pu',
                              'info' : {'buf_size' : 1e100,
                                        'efficiencies' : {'item' : {'comp': 'Pu239', 
                                                                    'eff': 0.99}}
                                       }
                             },
                             {'commod' : 'UOX_RU',
                              'info' : {'buf_size' : 1e100,
                                        'efficiencies' : {'item' : {'comp': 'U238', 
                                                                    'eff': 0.99}}
                                       }
                             }
                            ]
                  }
lwr_sep = {'name' : 'UOXSep',
           'config' : {'Separations' : {'leftover_commod' : 'Waste',
                                        'throughput' : 83333.3333,
                                        'feedbuf_size' : 107537,
                                        # can't use feed recipe unless we split into 2 sep facilities for each commod
                                        'feed_commods' : {'val' : ['CooledSpentUOX_Non', 'CooledSpentUOX_Add']},
                                        'feed_commod_prefs' : {'val' : [1, 2]},
                                        'streams' : lwr_sep_streams
                                       }
                      }
          }

In [7]:
# SFR Additions

mix_pu = {'info' : {'mixing_ratio' : 0.0900, 
                    'buf_size' : 1e8},
          'commodities' : {'item' : [{'commodity' : 'UOX_Pu',
                                      'pref' : 1},
                                     {'commodity' : 'FF_Pu',
                                      'pref' : 1}, # AFRC has 2 here
                                    ]
                          }
         }
mix_ru = {'info' : {'mixing_ratio' : 0.7100,
                    'buf_size' : 1e8
                   }, 
          'commodities' : {'item' : [{'commodity' : 'UOX_RU',
                                      'pref' : 1},
                                     {'commodity' : 'FF_RU',
                                      'pref' : 1}
                                    ]
                          }
         }
mix_natu = {'info' : {'mixing_ratio' : 0.1995,
                      'buf_size' : 1e8
                     }, 
            'commodities' : {'item' : [{'commodity' : 'StoredDepU',
                                        'pref' : 2},
                                       {'commodity' : 'NatU',
                                        'pref' : 1}
                                      ]
                            }
           }
mix_non_iso = {'info' : {'mixing_ratio' : 0.0005,
                         'buf_size' : 1e8
                        }, 
               'commodities' : {'item' : {'commodity' : 'NonIsos',
                                           'pref' : 1}
                               }
              }
mix_add_iso = {'info' : {'mixing_ratio' : 0.0005,
                         'buf_size' : 1e8
                        }, 
               'commodities' : {'item' : {'commodity' : 'AddIsos',
                                           'pref' : 1}
                               }
              }
sfr_mix_no232 = {'name' : 'FFMixNon',
                 'config' : {'Mixer' : {'in_streams' : {'stream' : [mix_pu, mix_ru, mix_natu, mix_non_iso]},
                                        'out_commod' : 'FF_Non',
                                        'throughput' : 1e10,
                                        'out_buf_size' : 1e20
                                       }
                            }
                }
sfr_mix_232 = {'name' : 'FFMixAdd',
               'config' : {'Mixer' : {'in_streams' : {'stream' : [mix_pu, mix_ru, mix_natu, mix_add_iso]},
                                      'out_commod' : 'FF_Add',
                                      'throughput' : 1e10,
                                      'out_buf_size' : 1e20
                                     }
                          }
              }
non_sfr_cool = {'name' : 'FFCoolNon',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentFF_Non'},
                                         'out_commods' : {'val' : 'CooledSpentFF_Non'},
                                         'in_recipe' : 'SpentFF_no232',
                                         'residence_time' : 81
                                        }
                           }
               }
add_sfr_cool = {'name' : 'FFCoolAdd',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentFF_Add'},
                                         'out_commods' : {'val' : 'CooledSpentFF_Add'},
                                         'in_recipe' : 'SpentFF_232',
                                         'residence_time' : 81
                                        }
                           }
               }
sfr_sep_streams = {'item' : [{'commod' : 'FF_Pu',
                              'info' : {'buf_size' : 1e100,
                                        'efficiencies' : {'item' : {'comp': 'Pu239', 
                                                                    'eff': 0.99}}
                                       }
                             },
                             {'commod' : 'FF_RU',
                              'info' : {'buf_size' : 1e100,
                                        'efficiencies' : {'item' : {'comp': 'U238', 
                                                                    'eff': 0.99}}
                                       }
                             }
                            ]
                  }
#other studies changed SepSFR values to 1e100
sfr_sep = {'name' : 'FFSep',
           'config' : {'Separations' : {'leftover_commod' : 'Waste',
                                        'throughput' : 83333.3333, 
                                        'feedbuf_size' : 107537,
                                        # can't use feed recipe unless we split into 2 sep facilities for each commod
                                        'feed_commods' : {'val' : ['CooledSpentFF_Non', 'CooledSpentFF_Add']},
                                        'feed_commod_prefs' : {'val' : [1, 2]},
                                        'streams' : sfr_sep_streams
                                       }
                      }
          }

In [8]:
sfr = {'name' : 'SFR', 
       'lifetime' : rxtr_life,
       'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['FF_Non', 'FF_Add']},
                                'fuel_outcommods' : {'val' : ['SpentFF_Non', 'SpentFF_Add']},
                                'fuel_inrecipes' :  {'val' : ['FF_no232', 'FF_232']},
                                'fuel_outrecipes' : {'val' : ['SpentFF_no232', 'SpentFF_232']},
                                'fuel_prefs' : {'val' : [1, 2]},
                                'cycle_time' : 14,
                                'refuel_time' : 0,
                                'assem_size' : 7490,
                                'n_assem_core' : 5,
                                'n_assem_batch' : 1,
                                'power_name' : 'PowerSFR',
                                'power_cap' : 400
                               }
                  }
      }

# Regions and Institutions:
1. Deploy Inst (Init Fleet)
2. Remainder of FC facilities
  - EG01 Manager Inst
  - Additional Facilities for Add Ramp-up (Manager Inst)
3. Growth Region
4. EG23 Deploy & Manager Insts (Separations and SFRs)

## 1. Init LWR Fleet (Deploy Inst)

In [9]:
init_lwr_prototypes = ['LWR' for x in range(0, n_init_rxtrs)]
n_builds = [1 for x in range(0, n_init_rxtrs)]
build_times = [x + 1 for x in range(0, 17) for y in range(0,6)]
del build_times[-3:-1]
# Lifetimes borrowed from previous EG scenario work presuming a 
# start year of 2000 and first decommission in 2015. Updated to 
# better stagger decommissioning.
old_lives = [181, 186, 191, 196, 
             201, 206, 211, 216, 221, 226, 231, 236, 241, 246, 251, 256, 261, 266, 271, 276, 281, 286, 291, 296,
             301, 306, 311, 316, 321, 326, 331, 336, 341, 346, 351, 356, 361, 366, 371, 376, 381, 386, 391, 396,
             401, 406, 411, 416, 421, 426, 431, 436, 441, 446, 451, 456, 461, 466, 471, 476, 481, 486, 491, 496,
             501, 506, 511, 516, 521, 526, 531, 536, 541, 546, 551, 556, 561, 566, 571, 576, 581, 586, 591, 596,
             601, 606, 611, 616, 621, 626, 631, 636, 641, 646, 651, 656, 661, 666, 671, 676
            ]
# Overwrite lifetimes to have decommission start 1.5 year in, 
# expecting a sim start of 2022
lifetimes = [x - 163 for x in old_lives]

assert len(init_lwr_prototypes) == n_init_rxtrs
assert len(n_builds) == n_init_rxtrs
assert len(build_times) == n_init_rxtrs
assert len(lifetimes) == n_init_rxtrs

In [10]:
init_fleet = {'name' : 'InitFleet', 
              'config' : {'DeployInst' : {'prototypes' :  {'val' : init_lwr_prototypes},
                                          'n_build' :     {'val' : n_builds},
                                          'build_times' : {'val' : build_times},
                                          'lifetimes' :   {'val' : lifetimes}
                                         }
                         }
             }

## 2a. EG01 FC Inst (Manager Inst)

In [ ]:
eg1_fc_prototypes = ['SourceNatU', 'SourceNonIsos', 'SourceAddIsos', 'Enrichment', 'StorageDepU', 
                     'UOXStrNon', 'UOXMixNon', 'UOXStrAdd', 'UOXMixAdd', 'LWR', 
                     'UOXCool', 'UOXStr', 'Waste'
                    ]

eg1_fc_inst = {'name' : 'FCInstEG01', 
               'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                                   {'number' : 1, 'prototype' : 'SourceNatU'},
                                                   {'number' : 1, 'prototype' : 'SourceNonIsos'},
                                                   {'number' : 1, 'prototype' : 'SourceAddIsos'},
                                                   {'number' : 1, 'prototype' : 'Enrichment'},
                                                   {'number' : 1, 'prototype' : 'StorageDepU'},
                                                   {'number' : 1, 'prototype' : 'StorageNon'},
                                                   {'number' : 1, 'prototype' : 'UOXStrNon'},
                                                   {'number' : 1, 'prototype' : 'UOXMixNon'},
                                                   {'number' : 1, 'prototype' : 'UOXStrAdd'},
                                                   {'number' : 1, 'prototype' : 'UOXMixAdd'},
                                                   {'number' : 1, 'prototype' : 'UOXCool'},
                                                   {'number' : 1, 'prototype' : 'UOXStr'},
                                                   {'number' : 1, 'prototype' : 'Waste'}]
                                       },
               'config' : {'ManagerInst' : {'prototypes' : {'val' : eg1_fc_prototypes}}}
              }

## 2b. Add-on Facilities for Add Ramp-up

In [ ]:
eg1_ramp_prototypes = ['StorageRampNon', 'StorageRampAdd', 
                       'Mixer50pctAdd', 'Mixer10pctAdd'
                      ]

t = intro_time

eg1_ramp_inst = {'name' : 'RampInstEG01', 
                 'config' : {'DeployInst' : {'prototypes' :  {'val' : eg1_ramp_prototypes},
                                             'n_build' :     {'val' : [1, 1, 1, 1]},
                                             'build_times' : {'val' : [t, t, t, t]}
                                            }
                            }
                }

## 3. Growth Region

In [29]:
lin_func_0 = {'type' : 'linear', 'params' : '0 100000'}
lin_func_1 = {'type' : 'linear', 'params' : '-75 0'}
lin_func_2 = {'type' : 'linear', 'params' : '75 0'}
lin_func_lwr = {'piece' : [{'start' : 18,
                            'function' : lin_func_0
                           },
                           {'start' : trans_time,
                            'function' : lin_func_1
                           }
                          ]
               }
lin_func_sfr = {'piece' : [{'start' : trans_time,
                            'function' : lin_func_2
                           }]
               }
growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'PowerLWR','piecewise_function' : lin_func_lwr},
                                                         {'commod' : 'PowerSFR','piecewise_function' : lin_func_sfr}]}}}

#orig_lin_func = {'piece' : [{'start' : 18,'function' : lin_func_1}]}
#growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'power','piecewise_function' : orig_lin_func}]}}}

## 4. EG23 FC Insts

In [12]:
eg1_23_fc_prototypes = ['SourceNatU', 'SourceNonIsos', 'SourceAddIsos', 'Enrichment', 'StorageDepU', 
                        'UOXStrNon', 'UOXMixNon', 'UOXStrAdd', 'UOXMixAdd', 'LWR', 
                        'UOXCoolNon', 'UOXCoolAdd', 'Waste', 'SFR'
                       ]

eg1_23_fc_inst = {'name' : 'FCInstEG01-23', 
                  'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                                      {'number' : 1, 'prototype' : 'SourceNatU'},
                                                      {'number' : 1, 'prototype' : 'SourceNonIsos'},
                                                      {'number' : 1, 'prototype' : 'SourceAddIsos'},
                                                      {'number' : 1, 'prototype' : 'Enrichment'},
                                                      {'number' : 1, 'prototype' : 'StorageDepU'},
                                                      {'number' : 1, 'prototype' : 'UOXStrNon'},
                                                      {'number' : 1, 'prototype' : 'UOXMixNon'},
                                                      {'number' : 1, 'prototype' : 'UOXStrAdd'},
                                                      {'number' : 1, 'prototype' : 'UOXMixAdd'},
                                                      {'number' : 1, 'prototype' : 'UOXCoolNon'},
                                                      {'number' : 1, 'prototype' : 'UOXCoolAdd'},
                                                      {'number' : 1, 'prototype' : 'Waste'}]
                                          },
                  'config' : {'ManagerInst' : {'prototypes' : {'val' : eg1_23_fc_prototypes}
                                              }
                             }
                 }

eg23_fc_prototypes = ['UOXSep', 'FFMixNon', 'FFMixAdd', 'FFCoolNon', 'FFCoolAdd', 'FFSep']

eg23_nbuilds = [1 for x in range(0, len(eg23_fc_prototypes))]
eg23_buildtimes = [trans_time for x in range(0, len(eg23_fc_prototypes))]

eg23_deploy = {'name' : 'EG23Deploy', 
               'config' : {'DeployInst' : {'prototypes' :  {'val' : eg23_fc_prototypes},
                                           'n_build' :     {'val' : eg23_nbuilds},
                                           'build_times' : {'val' : eg23_buildtimes}
                                          }
                          }
              }

#eg23_fc_inst = {'name' : 'FCInstEG23',
#                'config' : {'ManagerInst' : {'prototypes' : {'val' : 'SFR'}}}
#               }

# Recipes
-  Here: 
  1. Depleted U
  2. Natural U
-  100 ppt Init U232 Additive Recipes (in recipe_100ppt.py):
  1. NonAdditive U Isotopes (U234)
  2. Additive U Isotopes (U232, U233, U234)
  3. Almost UOX NonAdditive Enr Ratio
  4. Almost UOX Additive Enr Ratio
  5. UOX without Additive
  6. UOX with Additive
  7. Spent UOX from #5
  8. Spent UOX from #6

In [13]:
dep_u = {'name' : 'DU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.0025}, 
                      {'id' : 'U238', 'comp' : 0.9975}]
        }
nat_u = {'name' : 'NU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.007110}, 
                      {'id' : 'U238', 'comp' : 0.992890}]
        }

In [14]:
# Recipes from 100 ppt U232 additive @ beginning of enrichment
from recipe_100ppt import (isos_no232, isos_232, enr_no232, enr_232, 
                           uox_no232, uox_10pct232, uox_50pct232, uox_232, 
                           spent_no232, spent_10pct232, spent_50pct232, 
                           spent_232, ff_no232, ff_232, spentff_no232, 
                           spentff_232
                          )

# Main Input File 

In [15]:
control = {'duration' : sim_duration, 
           'startmonth' : 1, 
           'startyear' : 2022,
           'dt' : 86400
          }

## EG01

In [ ]:
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Enrichment'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                       ]
             }

In [ ]:
# full additive availability at intro
full_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, eg1_fc_inst]
              }
full_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : full_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, 
                                          lwr_full, lwr_cool, lwr_store, sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_232, spent_no232, spent_232
                                       ]
                           }
           }

In [ ]:
# ramp up additive availability at intro
ramp_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, eg1_fc_inst, eg1_ramp_inst]
              }
ramp_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : ramp_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, 
                                          store_pct_no232, store_pct_232, mix_50pct232, mix_10pct232, 
                                          lwr_ramp, lwr_cool, lwr_store, sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_10pct232, uox_50pct232, uox_232,
                                        spent_no232, spent_10pct232, spent_50pct232, spent_232
                                       ]
                           }
           }

## EG01 --> 23

In [30]:
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Enrichment'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                      ]
             }


# full additive availability at intro
full_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, eg1_23_fc_inst, eg23_deploy]
              }
full_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : full_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, lwr_full, 
                                          non_lwr_cool, add_lwr_cool, lwr_sep, sfr_mix_no232, sfr_mix_232, 
                                          sfr, sfr_sep, non_sfr_cool, add_sfr_cool, eg23_sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_232, spent_no232, spent_232, ff_no232, ff_232, 
                                        spentff_no232, spentff_232
                                       ]
                           }
           }

In [31]:
filebase = 'test_py'

in_file = filebase + '.py'
sim_file = '../output/' + filebase + '.sqlite'

with open(in_file, 'w') as file: 
    file.write('SIMULATION = ' + str(full_sim))
subprocess.run(['rm', sim_file])
subprocess.run(['cyclus', in_file, '-o', sim_file])

CompletedProcess(args=['cyclus', 'test_py.py', '-o', '../output/test_py.sqlite'], returncode=0)

# Set of Simulations 

List of Simulation Scenarios (24):

If this is done with flat power plus 1% growth in power, doubles to 48 simulations

- EG Scenarios
  1. 01
  2. 01-23
  3. 01-29
- Init Additive Concentration
  1. 100ppt
  2. ???pp?
- Date
  1. beginning of transition
  2. later in transition
- Rate
  1. full availability
  2. ramp up availability

In [19]:
# File Names:
for eg in ['01', '23', '29']:
    for ppx in ['100ppt', '100ppb']:
        for date in ['05yr', '15yr']:
            for rate in ['full', 'ramp']:
                file = eg + '_' + ppx + '_' + date + '_' + rate
                print(file)

01_100ppt_05yr_full
01_100ppt_05yr_ramp
01_100ppt_15yr_full
01_100ppt_15yr_ramp
01_100ppb_05yr_full
01_100ppb_05yr_ramp
01_100ppb_15yr_full
01_100ppb_15yr_ramp
23_100ppt_05yr_full
23_100ppt_05yr_ramp
23_100ppt_15yr_full
23_100ppt_15yr_ramp
23_100ppb_05yr_full
23_100ppb_05yr_ramp
23_100ppb_15yr_full
23_100ppb_15yr_ramp
29_100ppt_05yr_full
29_100ppt_05yr_ramp
29_100ppt_15yr_full
29_100ppt_15yr_ramp
29_100ppb_05yr_full
29_100ppb_05yr_ramp
29_100ppb_15yr_full
29_100ppb_15yr_ramp
